In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [84]:
from datetime import datetime
from io import StringIO
import re

In [2]:
url = r'https://en.wikipedia.org/wiki/List_of_American_films_of_2021'
r = requests.get(url)
r.status_code

200

In [3]:
soup1 = BeautifulSoup(r.content, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

In [87]:
tables = soup2.find_all('table')

In [135]:
title_link = []
for_range = len(tables[3].find_all('tr')) -1

for i in range(for_range):
    i += 1
    messy = tables[3].find_all('tr')[i].find_all('td')[-4]
    title_link.append(messy.find('a'))

title_link[0]

<a href="/wiki/Shadow_in_the_Cloud" title="Shadow in the Cloud">
                Shadow in the Cloud
               </a>

In [137]:
title_link = list(map(str, title_link))

In [139]:
link = []
title = []

pattern = r'="(.+)"\s'
pattern2 = r'e="(.+)"'

for i in title_link:
    link.append(re.findall(pattern, i)[0])
    title.append(re.findall(pattern2, i)[0])

link[:5]

['/wiki/Shadow_in_the_Cloud',
 '/wiki/Hacksaw_(film)',
 '/wiki/Dr._Bird%27s_Advice_for_Sad_Poets',
 '/wiki/The_White_Tiger_(2021_film)',
 '/wiki/Locked_Down_(film)']

In [142]:
df2 = pd.DataFrame({'Link':link, 'Title':title})
df2.tail()

,Link,Title
63,/wiki/Nobody_(2021_film),Nobody (2021 film)
64,/wiki/Bad_Trip_(film),Bad Trip (film)
65,/wiki/The_Seventh_Day_(2021_film),The Seventh Day (2021 film)
66,/wiki/Senior_Moment,Senior Moment
67,/wiki/Godzilla_vs._Kong,Godzilla vs. Kong


In [140]:
df = pd.read_html(StringIO(str(tables[3])))
df[0].tail()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
63,M A R C H,26,Nobody,Universal Pictures / Perfect World Pictures ...,Ilya Naishuller (director); Derek Kolstad (...,[54]
64,M A R C H,26,Bad Trip,Netflix / Orion Pictures,Kitao Sakurai (director/screenplay); Dan Curry...,NaN
65,M A R C H,26,The Seventh Day,Vertical Entertainment / Voltage Pictures,Justin P. Lange (director/screenplay); Guy Pe...,[55]
66,M A R C H,26,Senior Moment,Screen Media Films,"Giorgio Serafini (director); Kurt Brungardt, C...",[56]
67,M A R C H,31,Godzilla vs. Kong,Warner Bros. Pictures / Legendary Entertainm...,"Adam Wingard (director); Eric Pearson , Ma...",[57]


In [144]:
pd.merge(df2, df[0], on='Title').isna().any()

Link                  False
Title                 False
Opening               False
Opening.1             False
Production company    False
Cast and crew         False
Ref.                   True
dtype: bool